In [1]:
%matplotlib inline
import json
import pandas as pd
from pathlib import Path

In [2]:
annotation_json_path = '/opt/ml/input/revised/annotation.json'
annotation_json = None
with Path(annotation_json_path).open(encoding='utf8') as handle:
    annotation_json = json.load(handle)

annotation_pd = pd.DataFrame(annotation_json)

print(annotation_pd.head(10))

                                                                                     images
0N8A5660.jpg                              {'paragraphs': {}, 'words': {'0': {'points': [...
14B9AA34-3F88-40E2-A3E8-6A2ED0CAE7E1.JPG  {'paragraphs': {}, 'words': {'0': {'points': [...
1559458822576-3.jpg                       {'paragraphs': {}, 'words': {}, 'chars': {}, '...
1595697184898-29.jpg                      {'paragraphs': {}, 'words': {'0': {'points': [...
1604828077798-0.jpg                       {'paragraphs': {}, 'words': {'0': {'points': [...
1607660335218-2.jpg                       {'paragraphs': {}, 'words': {'0': {'points': [...
1607662862521-14.jpg                      {'paragraphs': {}, 'words': {}, 'chars': {}, '...
1607662862521-9.jpg                       {'paragraphs': {}, 'words': {'0': {'points': [...
1611555922111.jpg                         {'paragraphs': {}, 'words': {'0': {'points': [...
1613386234855-7.jpg                       {'paragraphs': {}, 'words': {'0': {'po

In [4]:
def has_4_coordinates(value:list) -> bool:
    result = True
    if(len(value) != 4):
       result = False
    
    return result

def has_minus_coordinates(value:list) -> bool:
    result = True
    value_flat = sum(value, [])
    if(sum(n < 0 for n in value_flat) > 0):
        result = False

    return result


def has_text(value:str) -> bool:
    result = True
    if(not value):
        result = False
    
    return result


def has_blank(value:str) -> bool:
    result = True
    if(' ' in value):
        result = False
    
    return result


annotation_modified_pd = annotation_pd.copy()
for index, row in annotation_pd.iterrows():
    #print(row.index)
    paragraphs_index = row['images']
    paragraphs = paragraphs_index['paragraphs']
    words = paragraphs_index['words']
    chars = paragraphs_index['chars']
    tags = paragraphs_index['tags']
    releations = paragraphs_index['relations']
    license_tag = paragraphs_index['license_tag']
    is_good_filter_result = True

    #words(좌표)가 0일 경우도 있음
    if(len(words) <= 0):
        is_good_filter_result = False
    else:
        for key, value in words.items():
            points = value['points']
            orientation = value['orientation']
            transcription = value['transcription']
            word_tags = value['word_tags']
            language = value['language']
            illegibility = value['illegibility']
            
            if( 
            ((has_4_coordinates(points) == True) and
            (has_minus_coordinates(points) == True) and
            (has_text(transcription) == True) and
            (has_blank(transcription) == True))
            != True):
                is_good_filter_result = False
            
            if(is_good_filter_result == False):
                break
    
    if(is_good_filter_result == False):
        annotation_modified_pd = annotation_modified_pd.drop(index)


print(annotation_pd.describe())
print(annotation_modified_pd.describe())

annotation_modified_pd.to_json("/opt/ml/input/revised/annotation_modified.json", orient='columns')

                                                   images
count                                                1650
unique                                               1542
top     {'paragraphs': {}, 'words': {}, 'chars': {}, '...
freq                                                  109
                                                   images
count                                                 719
unique                                                719
top     {'paragraphs': {}, 'words': {'0': {'points': [...
freq                                                    1
